In [1]:
# import json
import numpy as np
import pandas as pd
import json

from matplotlib.colors import Normalize
from matplotlib import cm
from itertools import product


#import plotly.plotly as ply
#import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline

from ipywidgets import HBox, VBox, Button, widgets


mapbox_access_token = 'pk.eyJ1IjoiYnVwZSIsImEiOiJjanc3ZnpmMDEwYm1vNDNsbW15bXh3Y2RlIn0.DFOGHx9u2JThJKBQo1zDMQ'

compare_years = 30

In [2]:
ub = pd.read_csv('test_ub.csv', index_col=0)
districts = ub['name_3'].values.tolist()
districts[0:43]= districts[1:44]

with open('ub_district1.json') as f:
    ub_districts = json.load(f)
 
    
xx = np.arange(1976,2100)
colors1 = ['lightslategray',] * len(xx)


rcp = ["rcp45", "rcp85"]
gcm = ["GFDL-ESM2M","GFDL-ESM2M_ISIMIP","MIROC5_ISIMIP", "MPI_REMO2009", "MPI_REMO2009_ISIMIP3","MPI-ESM-LR"]

input_rcp = "M:/matbox/UB/tas_rcp_year.npy"
yyy = np.load(input_rcp)

yyy[:,:,:,0:43]= yyy[:,:,:,1:]

yyy[3,0,30:124,:] += 3
yyy[4,0,30:124,:] += 3

# store - gcm ( 6 gcms) ,rcps (rcp45,rcp85), years (1976-2099), 43 districts, 41 = Upper Bhima
yy = yyy[0,0,:,41] # GFDL-ESM2M, rcp45, 1976-2099 ,upper Bhima  


In [3]:
zz = ub.t1.to_numpy() 
no_district = len(zz)
q = np.zeros(no_district)

for i in range(no_district):
    yy = yyy[0,0,:,i]
    m = yy[:30].mean()
    mact = yy[124-compare_years:124].mean()
    #print (m,mact)
    #q[i]= round(100 * mact / m,1)
    q[i] = round(mact - m,2)




#print (q)
# Hot, Rainbow
    
data1 = go.Choroplethmapbox(name ="Upper Bhima",geojson=ub_districts, locations=ub.index,z=q,
                            colorscale="Portland", zmin=-0.5, zmax=+3, marker_opacity=0.5, hovertext=districts)
#layout1 = dict(mapbox_style="carto-positron", mapbox_accesstoken=mapbox_access_token,
#                  mapbox_zoom = 6, mapbox_center = {"lat": 18.0, "lon": 75.0}, mapbox_pitch = 0,
#               title = 'Upper Bhima Basin - Climate Analysis')

mapbox1 = dict(accesstoken=mapbox_access_token,
        style="mapbox://styles/bupe/cjw7g6cax0hem1cqy9vc9vln4",
        center=go.layout.mapbox.Center(lat=18,lon=75),
        pitch=0,  zoom=6)

anno  =[dict( x=0.6,y=4.3,xref="x", yref="y",
            text="Upper Bhima")]
             
layout1 = dict(mapbox = mapbox1, 
               title = 'Upper Bhima Basin - Climate Analysis',
              dragmode = False)


fig1 = go.FigureWidget(go.Figure(data=[data1],layout = layout1))

#fig1.data[0].zmax=10
#anno  =[dict( x=1,y=5,xref="x", yref="y",
#            text="Upper Bhima")]
#fig1.update_layout(annotations=anno)

#fig1.show(config={'scrollZoom': False})
#fig1.layout.title = 'Upper Bhima Basin - Climate Analysis'
fig1


FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'],
              …

In [4]:
# linear trend

yy = yyy[0,0,:,40]

print (xx.shape,yy.shape)
m,b =np.polyfit(xx,yy,1)
m1 = yy[:30].mean()
m2 = yy[124-compare_years:124].mean()

colorbar = np.where(yy > m1,"darkred","lightslategray")
colorbar = np.where(yy > m1+1.,"crimson",colorbar).tolist()

data2a = go.Bar(name='avg. Temp', x=xx, y=yy)
data2b = go.Scatter(x=xx,y= xx*m+b, name="Trend")
#data2c = go.Scatter(x=xx[:30],y= xx[:30]*0+m1, name="Mean")
#data2d = go.Scatter(x=xx[124-compare_years:124],y= xx[39-compare_years:39]*0+m2, marker_color ="black", name="Mean period", mode='lines')


layout2 = dict(title = 'Station 1')
#data2a.marker.color=colorbar
fig2 = go.FigureWidget(go.Figure(data=[data2a,data2b],layout=layout2))
fig2.update_yaxes(range=[yyy[0].min(), yyy[0].max()])

#fig2.update_layout(legend_title = 'Temp [deg C]')
fig2.update_layout(legend=dict(x= 0, y=1.0))
fig2
#xx[0:38]


(124,) (124,)


FigureWidget({
    'data': [{'name': 'avg. Temp',
              'type': 'bar',
              'uid': 'bae080e0-…

In [6]:

map = fig1.data[0]
bar1 = fig2.data[0]
bar2 = fig2.data[1]

# trigger time series district, start year
# trigger gcm, rcp, district, start year
trigger = [0,0,0,2099]


def update_bar(trace, maps, selector):
    #print (trace, points,selector)
    inds = maps.point_inds[0] 
    trigger[2] = inds  # districts

    #c = list(bar.marker.color)
    #with fig2.batch_update():
        #bar.marker.color = color1[inds]
    y = yyy[trigger[0],trigger[1],:,trigger[2]]
    bar1.y = y
    m,b =np.polyfit(xx,y,1)
    m1 = y[:30].mean()
    bar2.y = xx*m+b

    xxmax = np.argwhere(xx==trigger[3])[0][0] 
    m2 = y[xxmax-compare_years:xxmax].mean()

    
   
    fig2.layout.title = 'District: ' + districts[inds]
    #map.z = map.z -20
    #print ("map:",trigger[0],trigger[1])

map.on_click(update_bar)  


In [7]:
gcm = ["GFDL-ESM2M    ","GFDL-ESM2M_ISIMIP    ","MIROC5_ISIMIP   ", "MPI_REMO2009    ", "MPI_REMO2009_ISIMIP3","MPI-ESM-LR    "]
analysis = ["RCP 4.5","RCP 8.5"]

# store - gcm ( 6 gcms) ,rcps (rcp45,rcp85), years (1976-2099), 43 districts, 41 = Upper Bhima

textbox1 = widgets.Dropdown(
    description='GCMs:',
    value='GFDL-ESM2M    ',
    options= gcm
)
textbox2 = widgets.Dropdown(
    description='RCPs:',
    value='RCP 4.5',
    options= analysis
)

years = widgets.IntSlider(
    value=2099,
    min=2099-compare_years*3,
    max=2099,
    step= compare_years,
    description='Years',
    continuous_update=False
)

container = widgets.VBox([textbox1, textbox2, years])


def response(change):
    #ii = textbox2.value
    trigger[0] = gcm.index(textbox1.value)
    trigger[1] = analysis.index(textbox2.value)
    #print(ii,i)map.z = map.z -40
    bar1.y = yyy[trigger[0],trigger[1],:,trigger[2]]
    bar1.name = gcm[trigger[0]]  
    
    fig1.layout.title = "Upper Bhima Basin - Temp: " + gcm[trigger[0]] + " " + analysis[trigger[1]]
  
    xxmax = np.argwhere(xx==trigger[3])[0][0]
    q = np.zeros(no_district)
    for i in range(no_district):
        yy = yyy[trigger[0],trigger[1],:,i]
        m = yy[:30].mean()
        mact = yy[xxmax-compare_years:xxmax].mean()
        #q[i]=round(100 * mact / m,1)
        q[i] = round(mact - m,2)
    map.z = q
    

def responseyear(change):
    #print (years.value)
    trigger[3] = years.value
    xxmax = np.argwhere(xx==years.value)[0][0]
    #print (xxmax,years.value)
    #print (xx[xxmax-5:xxmax])
    
    q = np.zeros(no_district)
    for i in range(no_district):
        yy = yyy[trigger[0],trigger[1],:,i]
        m = yy[:30].mean()
        mact = yy[xxmax-compare_years:xxmax].mean()
        #q[i]=round(100 * mact / m,1)
        q[i] = round(mact - m,2)
    map.z = q
 
    y = yyy[trigger[0],trigger[1],:,trigger[2]]
    m2 = y[xxmax-compare_years:xxmax].mean()
    #bar4.x = xx[xxmax-compare_years:xxmax]
    #bar4.y = yyy[trigger[0],trigger[1],xxmax-compare_years:xxmax]*0+m2
    #print (q)
    
    
textbox1.observe(response, names="value")    
textbox2.observe(response, names="value")
years.observe(responseyear, names="value")

In [8]:
#dashboard = VBox([HBox([f1, f2]), button])
#dashboard = HBox([container,fig1, fig2])
#dashboard = HBox([fig1, VBox([fig2,None])])
d1 = VBox([container, fig2])
d2 = HBox([fig1,d1])
d2



    'data': [{'colorscale': [[0.0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'],…